In [ ]:
!pip install mesa

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [ ]:
cmap = matplotlib.cm.get_cmap('viridis', 7)
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([50/256, 50/256, 50/256, 1])   # Calle en gris
cmap[1] = np.array([256/256, 256/256, 0/256, 1])  # Semáforo en amarillo
cmap[2] = np.array([256/256, 0/256, 0/256, 1])    # Semáforo en rojo
cmap[3] = np.array([0/256, 256/256, 0/256, 1])    # Semáforo en verde
cmap[4] = np.array([0/256, 50/256, 100/256, 1])   # Carro azul
cmap[5] = np.array([230/256, 100/256, 20/256, 1])  # Carro naranja
cmap[6] = np.array([96/256, 170/256, 70/256, 1])  # Pasto verde

new_cmap = matplotlib.colors.ListedColormap(cmap)

street = [[6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6],
          [6, 6, 6, 6, 6, 6, 0, 1, 0, 6, 6, 6, 6, 6, 6]]

In [ ]:
class CarAgent(Agent):
    """
    Representa un agente en el modelo.
    """
    def __init__(self, unique_id, model, route):
        """
        Crea un agente carro en una posicion al azar y se le asigna su direccion y semaforo
        """
        directions = [(0, -1), (-1, 0), (0, 1), (1, 0)]
        super().__init__(unique_id, model)
        self.direction = directions[route]
        self.traffic_light = self.model.traffic_lights[route]
        self.out_of_bounds = False
        self.next_pos = None
    
    def step(self):
        """
        Este metodo calcula la proxima posicion del carro, y checa las condiciones para 
        detenerse. 
        """
        self.next_pos = (self.pos[0] + self.direction[0], self.pos[1] + self.direction[1])

        # Si el agente se sale del grid, se marca como fuera de los límites.
        if self.next_pos[0] < 0 or self.next_pos[0] > 14 or self.next_pos[1] < 0 or self.next_pos[1] > 14:
            self.out_of_bounds = True
            return

        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)

        # live_neighbours = np.sum([neighbour.live for neighbour in neighbours])

        # checamos si hay un carro en frente
        for neighbor in neighbours:
            if isinstance(neighbor, CarAgent):
                if (neighbor.pos[0] - self.pos[0], neighbor.pos[1] - self.pos[1]) == self.direction:
                    self.next_pos = self.pos
        
        # checamos si el carro está en un semáforo
        in_crossing = False
        #iteramos entre los valores de la dirección
        for i in range (2):
            # checamos cual de los valores no es 0
            if self.direction[i]:
                # si i es 1, el carro tiene direccion horizontal
                if i:
                    # checamos si el carro está en un cruce
                    if self.model.grid.width//2 - self.direction[i] * 2 == self.pos[i]:
                        in_crossing = True
                # si i es 0, el carro tiene direccion vertical
                else:
                    # checamos si el carro está en un cruce
                    if self.model.grid.height//2 - self.direction[i] * 2 == self.pos[i]:
                        in_crossing = True
        
        # si el carro esta en un cruce y el semáforo está en rojo, el carro no puede pasar
        if in_crossing and not self.traffic_light.light:
            self.next_pos = self.pos      

    def advance(self):
        """
        Define el nuevo estado calculado del método step.
        """
        if self.out_of_bounds:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            self.model.grid.move_agent(self, self.next_pos)

class TrafficLightAgent(Agent):
    '''
    Representa un semaforo en el modelo
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.light = 0

class StreetAgent(Agent):
    '''
    Representa una calle en el modelo
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.color = street[self.unique_id[0][0]][self.unique_id[0][1]]
            
class IntersectionModel(Model):
    """
    Define el modelo del juego de la vida.
    """
    def __init__(self, width, height):
        self.grid = MultiGrid(width, height, False)
        self.traffic_lights = []
        self.traffic_light_on = 0
        self.starting_pos = [(width//2 - 1, height - 1), (width - 1,height//2 + 1), (width//2 + 1, 0), (0, height//2 - 1)]
        self.step_count = 0
        self.schedule = SimultaneousActivation(self)
        
        # Se crean los agentes calle para el fondo del modelo
        for (content, x, y) in self.grid.coord_iter():
            a = StreetAgent(((x, y), "Street"), self)
            self.grid.place_agent(a, (x, y))

        # Se crean los agentes semáforos
        trafficLightCoords = [(width//2, height//2 - 2), (width//2 - 2, height//2), (width//2, height//2 + 2), (width//2 + 2, height//2)]
        for i in range (4):
            a = TrafficLightAgent(((trafficLightCoords[i]), "TrafficLight"), self)
            self.traffic_lights.append(a)
            self.grid.place_agent(a, trafficLightCoords[i])
        # Se inicializa un semaforo en verde
        self.traffic_lights[0].light = 1

        # Aquí definimos el colector de datos para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": self.get_grid}
        )
    
    def step(self):
        """
        En cada paso el colector toma la información que se definió y almacena el grid para luego
        graficarlo.
        """
        # Cambiar color de semáforo
        if not self.step_count % 10:
            self.traffic_lights[self.traffic_light_on].light = 0
            self.traffic_light_on = (self.traffic_light_on + 1) % 4
            self.traffic_lights[self.traffic_light_on].light = 1


        # Crear agente carro en la calle
        if not self.step_count % 5:
            route = np.random.randint(0, 4)
            a = CarAgent((self.step_count, "Car"), self, route)
            self.grid.place_agent(a, self.starting_pos[route])
            self.schedule.add(a)

        self.step_count += 1

        self.datacollector.collect(self)
        self.schedule.step()

    def get_grid(self):
        """
        Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
        :param model: El modelo del cual obtener el grid.
        :return: Matriz con la información del grid del agente.
        """

        # Generamos la grid para contener los valores
        grid = np.zeros((self.grid.width, self.grid.height))

        # Asignamos una celda a cada uno de los elementos de la grilla
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            for agent in cell_content:
                if isinstance(agent, CarAgent):
                    grid[x][y] = 4
                elif isinstance(agent, TrafficLightAgent):
                    grid[x][y] = agent.light + 2
                elif isinstance(agent, StreetAgent):
                    grid[x][y] = agent.color
                

        return grid

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 15

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 200

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = IntersectionModel(GRID_SIZE, GRID_SIZE)
for i in range(NUM_GENERATIONS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(
    datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.065179


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()


In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim